In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

# BINARIA

In [ ]:
# Cargar los datos (reemplaza 'nombre_del_archivo.csv' con el nombre de tu archivo de datos)
data = pd.read_csv('nombre_del_archivo.csv')

# Cargar los datos (reemplaza 'nombre_del_archivo.csv' con el nombre de tu archivo de datos)
data = pd.read_csv('nombre_del_archivo.csv')

# Paso 2: Dividir los datos en características (X) y etiquetas (y)
X = data.drop('etiqueta', axis=1)
y = data['etiqueta']

# Paso 3: Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 4: Crear un pipeline con modelos para comparar y seleccionar
models = [('Logistic Regression', LogisticRegression()),
          ('Support Vector Machine', SVC(probability=True)),
          ('Random Forest', RandomForestClassifier())]

results = []
names = []

for name, model in models:
    pipeline = Pipeline([('scaler', StandardScaler()), (name, model)])
    
    # Ajuste de hiperparámetros utilizando GridSearchCV
    if name == 'Logistic Regression':
        param_grid = {'Logistic Regression__C': [0.1, 1, 10],
                      'Logistic Regression__solver': ['lbfgs', 'liblinear']}
    elif name == 'Support Vector Machine':
        param_grid = {'Support Vector Machine__C': [0.1, 1, 10],
                      'Support Vector Machine__kernel': ['linear', 'rbf']}
    elif name == 'Random Forest':
        param_grid = {'Random Forest__n_estimators': [50, 100, 200],
                      'Random Forest__max_depth': [None, 10, 20]}
    
    grid_search = GridSearchCV(pipeline, param_grid, cv=3)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    
    # Entrenar el modelo por minibatch (opcional)
    if hasattr(model, 'partial_fit'):
        batch_size = 32
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]
            best_model.partial_fit(X_batch, y_batch, classes=np.unique(y))
    else:
        best_model.fit(X_train, y_train)
    
    # Evaluar el modelo en el conjunto de prueba
    y_pred = best_model.predict(X_test)
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]
    
    # Métricas de rendimiento
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    results.append(roc_auc)
    names.append(name)
    
    print(f"Modelo: {name}")
    print("Mejores hiperparámetros:", grid_search.best_params_)
    print("ROC AUC:", roc_auc)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print("-" * 50)
# Paso 5: Comparar y seleccionar el mejor modelo
best_model_name = names[np.argmax(results)]
best_model = [model[1] for model in models if model[0] == best_model_name][0]

print(f"Mejor Modelo: {best_model_name}")

# Paso 6: Entrenar el mejor modelo en el conjunto de entrenamiento completo
best_pipeline = Pipeline([('scaler', StandardScaler()), (best_model_name, best_model)])
best_pipeline.fit(X_train, y_train)

# Paso 7: Evaluar el mejor modelo en el conjunto de prueba
y_pred = best_pipeline.predict(X_test)
y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]

print(f"Mejor Modelo: {best_model_name}")
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Paso 8: Gráficas para observar el rendimiento del mejor modelo
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'{best_model_name} (area = {roc_auc_score(y_test, y_pred_proba):.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

# MULTICLASE

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [ ]:
# Cargar los datos (reemplaza 'nombre_del_archivo.csv' con el nombre de tu archivo de datos)
data = pd.read_csv('nombre_del_archivo.csv')

# Paso 2: Dividir los datos en características (X) y etiquetas (y)
X = data.drop('etiqueta', axis=1)
y = data['etiqueta']

# Paso 3: Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 4: Crear un pipeline con modelos para comparar y seleccionar
models = [('Logistic Regression', LogisticRegression(multi_class='ovr')),
          ('Support Vector Machine', SVC(probability=True)),
          ('Random Forest', RandomForestClassifier())]

results = []
names = []

for name, model in models:
    pipeline = Pipeline([('scaler', StandardScaler()), (name, model)])
    
    # Ajuste de hiperparámetros utilizando GridSearchCV
    if name == 'Logistic Regression':
        param_grid = {'Logistic Regression__C': [0.1, 1, 10],
                      'Logistic Regression__solver': ['lbfgs', 'liblinear']}
    elif name == 'Support Vector Machine':
        param_grid = {'Support Vector Machine__C': [0.1, 1, 10],
                      'Support Vector Machine__kernel': ['linear', 'rbf']}
    elif name == 'Random Forest':
        param_grid = {'Random Forest__n_estimators': [50, 100, 200],
                      'Random Forest__max_depth': [None, 10, 20]}
    
    grid_search = GridSearchCV(pipeline, param_grid, cv=3)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    
    # Entrenar el modelo por minibatch (opcional)
    if hasattr(model, 'partial_fit'):
        batch_size = 32
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]
            best_model.partial_fit(X_batch, y_batch, classes=np.unique(y))
    else:
        best_model.fit(X_train, y_train)
    
    # Evaluar el modelo en el conjunto de prueba
    y_pred = best_model.predict(X_test)
    y_pred_proba = best_model.predict_proba(X_test)
    
    # Métricas de rendimiento
    roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
    results.append(roc_auc)
    names.append(name)
    
    print(f"Modelo: {name}")
    print("Mejores hiperparámetros:", grid_search.best_params_)
    print("ROC AUC:", roc_auc)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print("-" * 50)

# Paso 5: Comparar y seleccionar el mejor modelo
best_model_name = names[np.argmax(results)]
best_model = [model[1] for model in models if model[0] == best_model_name][0]

print(f"Mejor Modelo: {best_model_name}")

# Paso 6: Entrenar el mejor modelo en el conjunto de entrenamiento completo
best_pipeline = Pipeline([('scaler', StandardScaler()), (best_model_name, best_model)])
best_pipeline.fit(X_train, y_train)

# Paso 7: Evaluar el mejor modelo en el conjunto de prueba
y_pred = best_pipeline.predict(X_test)
y_pred_proba = best_pipeline.predict_proba(X_test)

print(f"Mejor Modelo: {best_model_name}")
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba, multi_class='ovr'))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Paso 8: Gráficas para observar el rendimiento del mejor modelo
# Las curvas ROC no son aplicables para clasificación multiclase, se puede usar la matriz de confusión
import seaborn as sns
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicciones')
plt.ylabel('Valores reales')
plt.title(f'Matriz de Confusión - {best_model_name}')
plt.show()

# Regresión

In [ ]:
# Paso 1: Importar las bibliotecas y cargar los datos
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# Cargar los datos (reemplaza 'nombre_del_archivo.csv' con el nombre de tu archivo de datos)
data = pd.read_csv('nombre_del_archivo.csv')

# Paso 2: Dividir los datos en características (X) y etiquetas (y)
X = data.drop('etiqueta', axis=1)
y = data['etiqueta']

# Paso 3: Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 4: Crear un pipeline con modelos para comparar y seleccionar
models = [('Linear Regression', LinearRegression()),
          ('Support Vector Machine', SVR()),
          ('Random Forest', RandomForestRegressor())]

results = []
names = []

for name, model in models:
    pipeline = Pipeline([('scaler', StandardScaler()), (name, model)])
    
    # Ajuste de hiperparámetros utilizando GridSearchCV
    if name == 'Linear Regression':
        param_grid = {'Linear Regression__fit_intercept': [True, False]}
    elif name == 'Support Vector Machine':
        param_grid = {'Support Vector Machine__C': [0.1, 1, 10],
                      'Support Vector Machine__kernel': ['linear', 'rbf']}
    elif name == 'Random Forest':
        param_grid = {'Random Forest__n_estimators': [50, 100, 200],
                      'Random Forest__max_depth': [None, 10, 20]}
    
    grid_search = GridSearchCV(pipeline, param_grid, cv=3)
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    
    # Entrenar el modelo por minibatch (opcional)
    if hasattr(model, 'partial_fit'):
        batch_size = 32
        for i in range(0, len(X_train), batch_size):
            X_batch = X_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]
            best_model.partial_fit(X_batch, y_batch)
    else:
        best_model.fit(X_train, y_train)
    
    # Evaluar el modelo en el conjunto de prueba
    y_pred = best_model.predict(X_test)
    
    # Métricas de rendimiento
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results.append((mse, r2))
    names.append(name)
    
    print(f"Modelo: {name}")
    print("Mejores hiperparámetros:", grid_search.best_params_)
    print("Mean Squared Error:", mse)
    print("R-squared:", r2)
    print("-" * 50)

# Paso 5: Comparar y seleccionar el mejor modelo
best_model_name = names[np.argmin([mse for mse, _ in results])]
best_model = [model[1] for model in models if model[0] == best_model_name][0]

print(f"Mejor Modelo: {best_model_name}")

# Paso 6: Entrenar el mejor modelo en el conjunto de entrenamiento completo
best_pipeline = Pipeline([('scaler', StandardScaler()), (best_model_name, best_model)])
best_pipeline.fit(X_train, y_train)

# Paso 7: Evaluar el mejor modelo en el conjunto de prueba
y_pred = best_pipeline.predict(X_test)

mse_best = mean_squared_error(y_test, y_pred)
r2_best = r2_score(y_test, y_pred)

print(f"Mejor Modelo: {best_model_name}")
print("Mean Squared Error:", mse_best)
print("R-squared:", r2_best)

# Paso 8: Gráficas para visualizar el rendimiento del mejor modelo (opcional)
# Por ejemplo, una gráfica de dispersión entre las etiquetas reales y las predicciones
plt.scatter(y_test, y_pred)
plt.xlabel('Etiquetas reales')
plt.ylabel('Predicciones')
plt.title('Gráfica de dispersión')
plt.show()